In [10]:
import pyLDAvis.gensim
import pyLDAvis
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import csv
from scipy.spatial import distance
import collections
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import numpy as np
import swifter  # Makes applying to datframe as fast as vectorizing
from gensim import models  # For TF-IDF, LDA
import gensim
from nltk.probability import FreqDist
import string
from nltk.corpus import stopwords
import glob
import os
import pandas as pd
import re
from pandas import Series, DataFrame
from textblob import TextBlob, Word
import nltk
from nltk.util import ngrams
import re
nltk.download('stopwords')
from scipy import sparse
from ast import literal_eval
# Visualization
plt.style.use('seaborn-paper')
%matplotlib inline

# LDA Visualization

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stop_words = ['a', 'aaa', 'about', 'above', 'account', 'activity', 'addition', 'affect', 
              'after', 'again', 'against', 'age', 'ain', 'alan', 'all', 'allow', 'almost',
              'along', 'along', 'also', 'although', 'always', 'am', 'america', 'american', 
              'among', 'amount', 'an', 'analysis', 'and', 'and/or', 'another', 'any', 'apa', 
              'appear', 'application', 'apply', 'approach', 'approximately', 'are', 'area', 
              'aren', "aren't", 'as', 'aspect', 'associate', 'association', 'assume', 'at', 
              'attempt', 'author', 'average', 'base', 'basis', 'be', 'because', 'become', 'been', 
              'before', 'behavior', 'being', 'below', 'best', 'between', 'both', 'but', 'by', 
              'calculate', 'calculation', 'can', 'case', 'cause', 'certain', 'change', 
              'characteristic', 'characterize', 'conclude', 'closely', 'common', 'compare', 
              'comparison', 'complete', 'complex', 'component', 'condition', 'consider', 
              'consist', 'consistent', 'constant', 'contain', 'control', 'copyright', 'could', 
              'couldn', "couldn't", 'd', 'data', 'database', 'day', 'decrease', 'define', 
              'demonstrate', 'depend', 'describe', 'detect', 'determine', 'develop', 'did', 'didn',
              "didn't", 'differ', 'difference', 'different', 'directly', 'discuss', 'distribution', 'do', 
              'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'done', 'down', 'due', 'during', 'e.g.', 
              'each', 'early', 'effect', 'effective', 'eight', 'either', 'employ', 'enough', 'especially',
              'establish', 'establish', 'estimate', 'etc', 'evaluate', 'even', 'evidence', 'examine', 
              'example', 'exhibit', 'exhibit', 'experiment', 'experimental', 'explain', 'factor', 'far', 
              'feature', 'few', 'find', 'finding', 'first', 'five', 'follow', 'for', 'form', 'formation', 
              'found', 'four', 'free', 'from', 'function', 'further', 'furthermore', 'general', 'give', 
              'good', 'great', 'group', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have',
              'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'high', 'highly', 
              'him', 'himself', 'his', 'hitherto', 'how', 'however', 'human', 'i', 'if', 'important', 
              'improved', 'in', 'inc', 'include', 'increase', 'increased', 'indicate', 'individual',
              'induce', 'influence', 'information', 'initial', 'interaction', 'into', 'is', 
              'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'kg', 'km', 'know', 'large', 'latter', 
              'least', 'less', 'level', 'likely', 'limit', 'line', 'll', 'low', 'm', 'ma', 'made', 
              'mainly', 'make', 'man', 'many', 'maximum', 'may', 'me', 'mean', 'measure', 
              'measurement', 'method', 'mg', 'might', 'mightn', "mightn't", 'ml', 'mm', 'monkey', 
              'month', 'more', 'most', 'mostly', 'mouse', 'much', 'multiple', 'must', 'mustn',
              "mustn't", 'my', 'myself', 'near', 'nearly', 'need', 'needn', "needn't", 'neither',
              'new', 'nih', 'nine', 'no', 'nor', 'normal', 'not', 'noticeable', 'now', 'number', 
              'o', 'observation', 'observe', 'obtain', 'obtained', 'occur', 'of', 'off', 'often', 
              'on', 'once', 'one', 'only', 'or', 'order', 'other', 'our', 'ours', 'ourselves', 
              'out', 'over', 'overall', 'own', 'parameter', 'part', 'particular', 'per', 'perform', 
              'perhaps', 'period', 'phase', 'physical', 'pmid', 'point', 'population', 'possible', 
              'potential', 'presence', 'present', 'press', 'previous', 'previously', 'probably', 
              'process', 'produce', 'product', 'property', 'propose', 'provide', 'psycinfo', 
              'publish', 'quantity', 'quite', 'rabbit', 'range', 'rat', 'rate', 'rather', 're', 
              'reaction', 'really', 'receive', 'recent', 'recently', 'record', 'ref', 'reference', 
              'regarding', 'relate', 'relation', 'relationship', 'relative', 'relatively', 'relevant', 
              'remain', 'report', 'represent', 'require', 'respectively', 'response', 'result', 
              'reveal', 'review', 'right', 'right reserved', 'role', 's', 'same', 'sample', 'second',
              'see', 'seem', 'seen', 'separate', 'set', 'seven', 'several', 'shan', "shan't", 'she', 
              "she's", 'should', "should've", 'shouldn', "shouldn't", 'show', 'showed', 'shown', 
              'shows', 'significant', 'significantly', 'similar', 'simple', 'since', 'six', 'size',
              'small', 'so', 'society', 'some', 'specific', 'state', 'strong', 'study', 'subject',
              'subsequent', 'such', 'suggest', 'support', 't', 'take', 'task', 'technique', 'ten', 
              'term', 'test', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
              'themselves', 'then', 'theory', 'there', 'therefore', 'these', 'they', 'this', 'those',
              'three', 'through', 'thus', 'time', 'to', 'together', 'too', 'total', 'two', 'type',
              'under', 'university', 'unknown', 'unlikely', 'until', 'up', 'upon', 'use', 'used', 
              'useful', 'using', 'usually', 'value', 'various', 'vary', 've', 'very', 'via', 'view', 
              'was', 'wasn', "wasn't", 'we', 'well', 'were', 'weren', "weren't", 'what', 'whatever', 
              'when', 'where', 'whereas', 'whether', 'which', 'while', 'who', 'whom', 'why', 'wiley', 
              'will', 'with', 'within', 'without', 'won', "won't", 'work', 'would', 'wouldn', 
              "wouldn't", 'y', 'yield', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours',
              'yourself', 'yourselves', 'involve', 'investigate', 'apparent', 'identify', 'assess']
           


def preprocess_text(doc, n_grams='one'):
    """
    Pre-processing using TextBlob: 
    tokenizing, converting to lower-case, and lemmatization based on POS tagging, 
    removing stop-words, and retaining tokens greater than length 2

    We can also choose to include n_grams (n = 1,2,3) in the final output

    Argument(s): 'doc' - a string of words or sentences.
                 'n_grams' - one: only unigrams (tokens consisting of one word each)
                           - two: only bigrams
                           - two_plus: unigrams + bigrams
                           - three: only trigrams 
                           - three_plus: unigrams + bigrams + trigrams

    Output: 'reuslt_singles' - a list of pre-processed tokens (individual words) of each sentence in 'doc'
            'result_ngrams' - a list of pre-processed tokens (including n-grams) of each sentence in 'doc'

    """

    blob = TextBlob(doc).lower()
#     lang = blob.detect_language()
#     print(lang)
#     if lang != 'en':
#         blob = blob.translate(to = 'en')

    result_singles = []

    tag_dict = {"J": 'a',  # Adjective
                "N": 'n',  # Noun
                "V": 'v',  # Verb
                "R": 'r'}  # Adverb

    # For all other types of parts of speech (including those not classified at all)
    # the tag_dict object maps to 'None'
    # the method w.lemmatize() defaults to 'Noun' as POS for those classified as 'None'

    for sent in blob.sentences:

        words_and_tags = [(w, tag_dict.get(pos[0])) for w, pos in sent.tags]
        lemmatized_list = [w.lemmatize(tag) for w, tag in words_and_tags]

        for i in range(len(lemmatized_list)):

            if lemmatized_list[i] not in stop_words and len(lemmatized_list[i].lower()) > 2 and not lemmatized_list[i].isdigit():
                result_singles.append(lemmatized_list[i].lower())

    result_bigrams = ['_'.join(x) for x in ngrams(result_singles, 2)]

    result_bigrams = [
        token for token in result_bigrams if token != 'psychological_association']

    result_trigrams = ['_'.join(x) for x in ngrams(result_singles, 3)]
    result_two_plus = result_singles + result_bigrams
    result_three_plus = result_singles + result_bigrams + result_trigrams

    if n_grams == 'one':
        result = result_singles
    elif n_grams == 'two':
        result = result_bigrams
    elif n_grams == 'three':
        result = result_trigrams
    elif n_grams == 'two_plus':
        result = result_two_plus
    elif n_grams == 'three_plus':
        result = result_three_plus

    return result

In [4]:
def get_frequency(processed_text_list):
    """
    Using a built-in NLTK function that generates tuples
    We get the frequency distribution of all words/n-grams in a tokenized list
    We can get the proportion of the the token as a fraction of the total corpus size  ----> N/A
    We can also sort these frequencies and proportions in descending order in a dictionary object ----> N/A

    Argument(s): 'processed_text_list' - A list of pre-processed tokens

    Output(s): freq_dict - A dictionary of tokens and their respective frequencies in descending order
    """
    # prop_dict - A dictionary of tokens and their respective proportions as a fraction of the total corpus
    # combined_dict - A dictionary whose values are both frequencies and proportions combined within a list
    # """

    word_frequency = FreqDist(word for word in processed_text_list)

#     sorted_counts = sorted(word_frequency.items(), key = lambda x: x[1], reverse = True)
#     freq_dict = dict(sorted_counts)
    freq_dict = dict(word_frequency)
#     prop_dict = {key : freq_dict[key] * 1.0 / sum(freq_dict.values()) for key, value in freq_dict.items()}
#     combined_dict = {key : [freq_dict[key], freq_dict[key] * 1.0 / sum(freq_dict.values())] for key, value in freq_dict.items()}

    return freq_dict  # , prop_dict, combined_dict

In [40]:
def merge_vocab_dictionary(vocab_column):
    """
    Takes any number of token frequency dictionaries and merges them while summing 
    the respective frequencies and then calculates the proportion of the the tokens 
    as a fraction of the total corpus size and saves to text and CSV files


    Argument(s): vocab_column - A column of dictionary objects

    Output(s): merged_combined_dict - A list object containing the frequencies of all
               merged dictionary tokens along with their respective proportions
    """

    merged_freq_dict = {}
    for dictionary in vocab_column:
        for key, value in dictionary.items():  # d.items() in Python 3+
            merged_freq_dict.setdefault(key, []).append(1)

    for key, value in merged_freq_dict.items():
        merged_freq_dict[key] = sum(value)

    sorted_merged_freq_dict = sorted(
        merged_freq_dict.items(), key=lambda x: x[1], reverse=True)

#     total_sum = sum(merged_freq_dict.values())
#     merged_prop_dict = {key : merged_freq_dict[key] * 1.0 / total_sum for key, value in merged_freq_dict.items()}
#     merged_combined_dict = {key : [merged_freq_dict[key], (merged_freq_dict[key] * 1.0 / total_sum)] for key, value in merged_freq_dict.items()}

    return sorted_merged_freq_dict

In [11]:
def fix_eval_issue(doc):
    return literal_eval(doc)

In [27]:
mcl_data = pd.read_csv(
    "/Users/shreya/Desktop/Theta_plus/MCL_ncf_abstract_top10_csv/ncf_20_top10_1_title_abstract.csv")

In [28]:
mcl_data.head()

,scp,title,abstract_text
0,14949207,Cleavage of structural proteins during the ass...,Using an improved method of gel electrophoresi...
1,84981840703,An Eight‐day Method for Screening Compounds ag...,SYNOPSIS. Intracardial inoculation of 1–10 mil...
2,17184389,A rapid and sensitive method for the quantitat...,A protein determination method which involves ...
3,23922515,Identification of the avian homologues of mamm...,Two mAb were produced against chicken T cells....
4,33751553722,Nitrogen to Protein Conversion Factor for Ten ...,The main two kinds of usual nitrogen to protei...


In [29]:
data_text = mcl_data.copy()
data_text['scp'] = data_text.astype('str')
data_text['processed_title'] = data_text['title'].swifter.apply(
    preprocess_text)
data_text['processed_abstract'] = data_text['abstract_text'].swifter.apply(
    preprocess_text)

data_text['processed_all_text'] = data_text['processed_title'] + data_text['processed_abstract']

In [30]:
data_text['processed_all_text_frequencies'] = data_text['processed_all_text'].swifter.apply(
    get_frequency)

In [31]:
data_all_text_frequency = merge_vocab_dictionary(
    data_text['processed_all_text_frequencies'])

In [41]:
print('Total number of articles: ', len(data_text))

max_freq_tokens = [i[0] for i in data_all_text_frequency[:50]]
max_freq_token_counts = [i[1] for i in data_all_text_frequency[:50]]
fig = plt.figure(figsize=(15, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.bar(max_freq_tokens, max_freq_token_counts)
plt.title('Top 50 Most Frequent Tokens in ncf_20_1_title')
plt.ylabel('Frequncy')
plt.xticks(rotation=90)
plt.show()

Total number of articles:  78


TypeError: unhashable type: 'slice'

In [ ]:
# def make_bow(doc):

#     bow_corpus = id2word_n_1.doc2bow(doc)

#     return bow_corpus

In [ ]:
# Create Dictionary
mcl_dictionary = gensim.corpora.Dictionary(
    data_text.processed_all_text.tolist())

# Create Corpus
mcl_all_text = data_text.processed_all_text.tolist()

mcl_corpus = [' '.join(text) for text in mcl_all_text]

# Term Document Frequency
mcl_bow_corpus = [mcl_dictionary.doc2bow(text) for text in mcl_all_text]

In [ ]:
num_topics = 1

# Build LDA model
mcl_lda = gensim.models.ldamodel.LdaModel(corpus=mcl_bow_corpus,
                                          id2word=mcl_dictionary,
                                          num_topics=num_topics,
                                          random_state=100,
                                          update_every=1,
                                          chunksize=100,
                                          passes=10,
                                          alpha='auto',
                                          per_word_topics=True)

In [ ]:
print(mcl_lda.print_topics(num_words=50))

In [ ]:
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(mcl_lda, mcl_bow_corpus, mcl_dictionary)
# vis

In [ ]:
def get_lda_tokens(lda_model=mcl_lda, num_tokens=30):
    """
    In current state, only works for num_topics = 1
    """

    x = lda_model.show_topics(num_words=num_tokens, formatted=False)

    topic_tokens = []
    for topics in x:
        for word, prob in topics[1]:
            topic_tokens.append(word)

    return topic_tokens

In [ ]:
# Cosine Similarities
def token_to_doc_similarity(doc, compare='lda', num_tokens=30):

    if compare == 'lda':
        compare_tokens = get_lda_tokens(mcl_lda, num_tokens)
    elif compare == 'frequency':
        compare_tokens = [i[0] for i in data_all_text_frequency[:num_tokens]]

    vectorizer = CountVectorizer()

    count_vec_compare = vectorizer.fit_transform([' '.join(compare_tokens)])
    count_vec_doc = vectorizer.transform([' '.join(doc)])

    cos_sim = cosine_similarity(count_vec_compare, count_vec_doc)

    return cos_sim[0][0]

In [ ]:
data_text['lda_similarity'] = data_text['processed_all_text'].swifter.apply(
    token_to_doc_similarity, args=('lda', 30))
data_text['max_frequency_similarity'] = data_text['processed_all_text'].swifter.apply(
    token_to_doc_similarity, args=('frequency', 30))

In [ ]:
print('The total number of articles is: ', len(data_text))
plt.figure(figsize=(10, 5))
plt.hist(data_text['lda_similarity'])
plt.suptitle('Distribution of Cosine Similarity Values')
plt.title('Between Articles and Top 30 LDA Tokens')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.show()

print('Number of articles with cos_sim > 0.3: ', len(
    data_text[data_text['lda_similarity'] > 0.3]))

In [ ]:
print('The total number of articles is: ', len(data_text))
plt.figure(figsize=(10, 5))
plt.hist(data_text['max_frequency_similarity'])
plt.suptitle('Distribution of Cosine Similarity Values')
plt.title('Between Articles and Top 30 Most Frequent Tokens')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.show()

print('Number of articles with cos_sim > 0.3: ', len(
    data_text[data_text['max_frequency_similarity'] > 0.2]))

In [ ]:
# LDA gives us:

# - A distribution over topics for each document
# - A distribution over words for each topics

In [ ]:
# data_text['article_bow'] = data_text['processed_all_text'].swifter.apply(make_bow)

In [ ]:
# for row_index, row in data_text_n_1[10:20].iterrows():
#     row = row.copy()
#     new_doc = [mcl_dictionary.doc2bow(row['processed_all_text'])]
# #     print(new_doc)
#     lda_result = gensim.models.ldamodel.LdaModel(corpus = new_doc,
#                                            id2word = mcl_dictionary,
#                                            num_topics = num_topics,
#                                            random_state = 100,
#                                            update_every = 1,
#                                            chunksize = 100,
#                                            passes = 10 ,
#                                            alpha = 'auto',
#                                            per_word_topics = True)


# #     x = lda_result.show_topics(num_words=30,formatted=False)

# #     doc_topics =[]
# #     for topics in x:
# #         for word, prob in topics[1]:
# #              doc_topics.append(word)

In [ ]:
mcl_total_cluster_corpus = [' '.join(mcl_corpus)]

In [ ]:
count_vectorizer = CountVectorizer()
mcl_total_cluster_count_mat = count_vectorizer.fit_transform(mcl_total_cluster_corpus)
mcl_doc_term_count_mat = count_vectorizer.transform(mcl_corpus)

prob_transformer = TfidfTransformer(norm='l1', use_idf=False, smooth_idf=False)

mcl_total_cluster_prob_mat = prob_transformer.fit_transform(mcl_total_cluster_count_mat)
mcl_doc_term_prob_mat = prob_transformer.fit_transform(mcl_doc_term_count_mat)

In [ ]:
mcl_total_cluster_count_mat

In [ ]:
mcl_doc_term_count_mat

In [ ]:
mcl_total_cluster_prob_mat

In [ ]:
mcl_doc_term_prob_mat

In [ ]:
data_text['doc_term_prob'] = mcl_doc_term_prob_mat.toarray().tolist()

In [ ]:
# distance.jensenshannon([1.0, 0.0, 0.0], [0.0, 1.0, 0.0])

In [ ]:
mcl_total_cluster_prob_vec = mcl_total_cluster_prob_mat.toarray().tolist()[0]


def calculate_jsd(prob_array, cluster_prob_vec=mcl_total_cluster_prob_vec):

    jsd = distance.jensenshannon(prob_array, cluster_prob_vec)

    return jsd

In [ ]:
data_text['JSD'] = data_text['doc_term_prob'].swifter.apply(calculate_jsd)

In [ ]:
data_text.head()

In [ ]:
data_text.describe()

In [ ]:
print('The total number of articles is: ', len(data_text))
plt.figure(figsize=(10, 5))
plt.hist(data_text['JSD'])
plt.suptitle('Distribution of JSD Values')
# plt.title('Between Articles and Top 30 Most Frequent Tokens')
plt.xlabel('JSD')
plt.ylabel('Frequency')
plt.show()

# print('Number of articles with cos_sim > 0.3: ', len(data_text[data_text['max_frequency_similarity']>0.2]))